# Tutorial 2: Parameters and convergence.

Content:
- Input preparation for LiF, F K-edge.
- Convergence with respect to number of bands in BSE part.
- Convergence with respect to number of bands in Screening part. 

Start by importing the SHORE library.

In [1]:
import shore

---
### Input preparation for LiF, F K-edge.

Load structure:

In [2]:
structure=shore.AtomicStructure('data/structures/lif.cif')

In [ ]:
structure.view()

Init photon and light objects:

In [4]:
photon=shore.Photon(polarization=[1,0,0],energy=dict(element="F", edge='K'))
light=shore.Light(photons=[photon])

Load parameters for tutorial purpose from file lif.param:

In [ ]:
matter=shore.Matter(structure=structure, load='data/inputs/lif.param')

What is the number of BSE bands (final state bands) and what is the number of bands used for screening? Hint:Check out `matter.show()`.

Create input file and come up with the name for it.

In [ ]:
ocean=shore.Input(name='',matter=matter,light=light)

Don't forget to init the server.

In [8]:
server=shore.RemoteServerManager(load='./jar/myserver.pkl')

Finally the calculation object is:

In [9]:
lif=shore.Calculation(server=server,input=ocean)

-----

### Convergence with respect to number of bands in BSE part.

<img src="data/figures/img6.png" alt="drawing" width="1000"/>

Initialize the dictionary were all calculations will be stored.

In [2]:
conv={}

For set of 10,15,20,25 bands create new calcualtion instances.

In [10]:
for nbands in [10,15,20,25]:
    conv[f'bse-{nbands}']=shore.Calculation(server=server, input=ocean.fork(name=f'bse-{nbands}',nbands=nbands))

Running new values:
```
for item in conv.values():
    item.run()
```
Checking status:
```
for k,v in conv.items():
    print(k)
    v.get_status()
```

Getting data from the cluster:
```
for k,v in conv.items():
    # print(k)
    v.sync()
```

Read results from `data/ref/LiF/` folder.

In [11]:
for key,value in conv.items():
    conv[key].res=shore.ResultsHandler().load(file=f'data/ref/LiF/conv_bse_bands/{key}.pkl')

Visualise results. 

In [12]:
import plotly.graph_objects as go

In [ ]:
fig=go.Figure()
for key,value in conv.items():
    conv[key].res.plot(fig=fig, element='F_', core_level='1s', site_number=1,lw=3, name=key, norm=False)
fig.update_xaxes(range=[-10,60])
fig.update_layout(title='LiF BSE bands')


What is the effect of the number of final state bands on the spectra?

Other things to consider:
- kmesh gs
- kmesh screen 
- kmesh bse

---
### Convergence with respect to number of bands in Screening part. 

<img src="data/figures/img7.png" alt="drawing" width="1000"/>

In [25]:
screen={}
for sbands in [10,25,50,75,100]:
    screen[sbands]=shore.Calculation(server=server, input=ocean.fork(name=f'screen-{sbands}',screen_nbands=sbands))

```
for sbands in [10,25,50,75,100]:
    screen[sbands].sync()

Syncing Files: 100%|██████████| 5/5 [00:00<00:00, 16.42file/s]
Syncing Files: 100%|██████████| 5/5 [00:00<00:00, 19.49file/s]
Syncing Files: 100%|██████████| 5/5 [00:00<00:00, 19.68file/s]
Syncing Files: 100%|██████████| 5/5 [00:00<00:00, 16.67file/s]
Syncing Files: 100%|██████████| 5/5 [00:00<00:00, 17.27file/s]
```

In [26]:
for key,value in screen.items():
    screen[key].res=shore.ResultsHandler().load(file=f'data/ref/LiF/conv_screen_bands/screen-{key}.pkl')

In [33]:
fig=go.Figure()
for key,value in screen.items():
    screen[key].res.plot(fig=fig, element='F_', core_level='1s', site_number=1, name=key, norm=False)
fig.update_layout(title='LiF SCREEN bands')

Why does the height of the first peak decrease as the number of bands used for screening increases?

-------